## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gmaps_api_key

# Configure gmaps API key
gmaps.configure(api_key=gmaps_api_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users/ryanj/github/World_Weather_Analysis/weather_database/weatherpy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind Speed,Current_Description,Country,Date
0,0,Bontang,0.1333,117.5000,76.568,90,100,4.22,moderate rain,ID,2021-11-28 17:12:16
1,1,Bluff,-46.6000,168.3333,51.728,75,78,7.48,broken clouds,NZ,2021-11-28 17:12:16
2,2,Riberalta,-10.9833,-66.1000,91.256,48,100,3.58,heavy intensity rain,BO,2021-11-28 17:12:16
3,3,Longyearbyen,78.2186,15.6401,8.438,66,40,7.20,scattered clouds,SJ,2021-11-28 17:12:17
4,4,Esperance,-33.8667,121.9000,72.752,42,100,5.77,overcast clouds,AU,2021-11-28 17:12:17


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = float(input('What is the maximum temperature: '))
min_temp = float(input('What is the minimum temperature: '))

What is the maximum temperature: 85
What is the minimum temperature: 70


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
temp_df = city_data_df.loc[(city_data_df.Max_Temp <= max_temp) & (city_data_df.Max_Temp >= min_temp)]

In [6]:
# 4a. Determine if there are any empty rows.
temp_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max_Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current_Description    0
Country                1
Date                   0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = temp_df.dropna()

In [8]:
clean_df.dtypes

City_ID                  int64
City                    object
Lat                    float64
Lng                    float64
Max_Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current_Description     object
Country                 object
Date                    object
dtype: object

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max_Temp", "Current_Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max_Temp,Current_Description,Lat,Lng,Hotel Name
0,Bontang,ID,76.568,moderate rain,0.1333,117.5000,
4,Esperance,AU,72.752,overcast clouds,-33.8667,121.9000,
10,Puerto Ayora,EC,76.964,overcast clouds,-0.7393,-90.3518,
13,Port Blair,IN,80.852,overcast clouds,11.6667,92.7500,
15,George Town,MY,78.728,moderate rain,5.4112,100.3354,
18,Rikitea,PF,76.388,clear sky,-23.1203,-134.9692,
27,Arraial Do Cabo,BR,80.456,scattered clouds,-22.9661,-42.0278,
29,Marsh Harbour,BS,72.464,broken clouds,26.5412,-77.0636,
30,Isangel,VU,77.450,moderate rain,-19.5500,169.2667,
33,Bogo,PH,78.656,broken clouds,11.0517,124.0055,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gmaps_api_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print('Hotel not found .... Skipping.')

Hotel not found .... Skipping.
Hotel not found .... Skipping.
Hotel not found .... Skipping.
Hotel not found .... Skipping.
Hotel not found .... Skipping.
Hotel not found .... Skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City                   0
Country                0
Max_Temp               0
Current_Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [12]:
# 8a. Create the output File (CSV)
clean_hotel_df = hotel_df.dropna()

# 8a_pt_2. 
clean_hotel_df.Max_Temp = clean_hotel_df.Max_Temp.map("{:,.1f}".format)

output_data_file = "C:/Users/ryanj/github/World_Weather_Analysis/vacation_search/hotel_data.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current_Description} and {Max_Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(25.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))